In [1]:
import pandas as pd
import numpy as np

In [64]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

In [52]:
from keras.layers import BatchNormalization

In [47]:
from sklearn.model_selection import train_test_split

In [13]:
dataset = pd.read_csv('Dataset/cleaned_dataset_with_labels.csv')

In [14]:
dataset = dataset[dataset.manual_label.notna()]

In [15]:
dataset.head()

,disease,drug,sentences,manual_label,distance,outlier_flag,avg_polarity,sentiment_rate_unsupervised,label_unsupervised
0,COVID-19,Hydroxychloroquine,She was confirmed to be SARS-CoV2 positive and...,0.0,10,1,0.242424,3705.013925,1
1,cancer,Teriflunomide,"Teriflunomide downregulation of MMP-9, epithel...",0.0,24,1,0.000000,3142.641881,1
2,hepatitis,Sofosbuvir,Sofosbuvir is an FDA-approved drug that is mai...,0.0,13,1,0.083333,1963.841772,1
3,cardiomyopathy,Acetylcholine,CHRM2 encodes muscarinic acetylcholine recepto...,0.0,26,1,-0.105556,2693.446820,1
4,cancer,Hydrogen,investigated the effects of Pd-Ni hydrogen sto...,1.0,6,1,0.030556,11067.976340,1


In [79]:
dataset.sentiment_rate_unsupervised.describe()

count       234.000000
mean      12455.145391
std       47629.701368
min         462.575643
25%        2479.939881
50%        4065.681753
75%        7445.448760
max      590452.710000
Name: sentiment_rate_unsupervised, dtype: float64

In [24]:
dataframe = dataset[['distance', 'avg_polarity', 'sentiment_rate_unsupervised', 'outlier_flag', 'manual_label']]

In [35]:
X = dataframe[['distance', 'avg_polarity', 'sentiment_rate_unsupervised', 'outlier_flag']].values

In [66]:
scalar = MinMaxScaler()
scalar.fit(X)
X = scalar.transform(X)

In [37]:
Y = dataframe['manual_label'].values

In [38]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [60]:
encoded_Y[0:10]

array([1, 1, 1, 1, 2, 0, 1, 2, 2, 2])

In [57]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [100]:
def binary_model():
    # create model
    model = Sequential()
    #model.add(BatchNormalization())
    model.add(Dense(8, input_dim=4, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dense(4, activation='tanh'))
    #model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [118]:
def junaed_model():
    model = keras.models.load_model('August/Model/junaed_model.h5')
    return model

In [63]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=2000, batch_size=50, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/2000
5/5 [==============================] - 0s 852us/step - loss: 1.4106 - accuracy: 0.2571
Epoch 2/2000
5/5 [==============================] - 0s 782us/step - loss: 1.4005 - accuracy: 0.2952
Epoch 3/2000
5/5 [==============================] - 0s 775us/step - loss: 1.3321 - accuracy: 0.2905
Epoch 4/2000
5/5 [==============================] - 0s 802us/step - loss: 1.3436 - accuracy: 0.2667
Epoch 5/2000
5/5 [==============================] - 0s 788us/step - loss: 1.2466 - accuracy: 0.3000
Epoch 6/2000
5/5 [==============================] - 0s 810us/step - loss: 1.2623 - accuracy: 0.2810
Epoch 7/2000
5/5 [==============================] - 0s 781us/step - loss: 1.2229 - accuracy: 0.2905
Epoch 8/2000
5/5 [==============================] - 0s 844us/step - loss: 1.2361 - accuracy: 0.2952
Epoch 9/2000
5/5 [==============================] - 0s 772us/step - loss: 1.2113 - accuracy: 0.3095
Epoch 10/2000
5/5 [==============================] - 0s 794us/step - loss: 1.1705 - accuracy: 0.3286

5/5 [==============================] - 0s 833us/step - loss: 0.9504 - accuracy: 0.5619
Epoch 163/2000
5/5 [==============================] - 0s 782us/step - loss: 0.9488 - accuracy: 0.5619
Epoch 164/2000
5/5 [==============================] - 0s 795us/step - loss: 0.9726 - accuracy: 0.5619
Epoch 165/2000
5/5 [==============================] - 0s 792us/step - loss: 0.9513 - accuracy: 0.5619
Epoch 166/2000
5/5 [==============================] - 0s 817us/step - loss: 0.9584 - accuracy: 0.5619
Epoch 167/2000
5/5 [==============================] - 0s 815us/step - loss: 0.9595 - accuracy: 0.5619
Epoch 168/2000
5/5 [==============================] - 0s 820us/step - loss: 0.9441 - accuracy: 0.5619
Epoch 169/2000
5/5 [==============================] - 0s 809us/step - loss: 0.9511 - accuracy: 0.5619
Epoch 170/2000
5/5 [==============================] - 0s 844us/step - loss: 0.9579 - accuracy: 0.5619
Epoch 171/2000
5/5 [==============================] - 0s 772us/step - loss: 0.9559 - accuracy: 0.

5/5 [==============================] - 0s 857us/step - loss: 0.9493 - accuracy: 0.5619
Epoch 323/2000
5/5 [==============================] - 0s 856us/step - loss: 0.9566 - accuracy: 0.5619
Epoch 324/2000
5/5 [==============================] - 0s 851us/step - loss: 0.9455 - accuracy: 0.5619
Epoch 325/2000
5/5 [==============================] - 0s 783us/step - loss: 0.9498 - accuracy: 0.5667
Epoch 326/2000
5/5 [==============================] - 0s 863us/step - loss: 0.9446 - accuracy: 0.5619
Epoch 327/2000
5/5 [==============================] - 0s 836us/step - loss: 0.9545 - accuracy: 0.5619
Epoch 328/2000
5/5 [==============================] - 0s 834us/step - loss: 0.9473 - accuracy: 0.5571
Epoch 329/2000
5/5 [==============================] - 0s 804us/step - loss: 0.9417 - accuracy: 0.5619
Epoch 330/2000
5/5 [==============================] - 0s 843us/step - loss: 0.9528 - accuracy: 0.5667
Epoch 331/2000
5/5 [==============================] - 0s 781us/step - loss: 0.9426 - accuracy: 0.

5/5 [==============================] - 0s 835us/step - loss: 0.9430 - accuracy: 0.5619
Epoch 483/2000
5/5 [==============================] - 0s 883us/step - loss: 0.9500 - accuracy: 0.5619
Epoch 484/2000
5/5 [==============================] - 0s 842us/step - loss: 0.9412 - accuracy: 0.5619
Epoch 485/2000
5/5 [==============================] - 0s 845us/step - loss: 0.9416 - accuracy: 0.5619
Epoch 486/2000
5/5 [==============================] - 0s 804us/step - loss: 0.9387 - accuracy: 0.5619
Epoch 487/2000
5/5 [==============================] - 0s 818us/step - loss: 0.9556 - accuracy: 0.5619
Epoch 488/2000
5/5 [==============================] - 0s 832us/step - loss: 0.9413 - accuracy: 0.5619
Epoch 489/2000
5/5 [==============================] - 0s 811us/step - loss: 0.9305 - accuracy: 0.5667
Epoch 490/2000
5/5 [==============================] - 0s 803us/step - loss: 0.9438 - accuracy: 0.5619
Epoch 491/2000
5/5 [==============================] - 0s 862us/step - loss: 0.9421 - accuracy: 0.

5/5 [==============================] - 0s 804us/step - loss: 0.9415 - accuracy: 0.5619
Epoch 643/2000
5/5 [==============================] - 0s 813us/step - loss: 0.9377 - accuracy: 0.5571
Epoch 644/2000
5/5 [==============================] - 0s 795us/step - loss: 0.9416 - accuracy: 0.5619
Epoch 645/2000
5/5 [==============================] - 0s 813us/step - loss: 0.9242 - accuracy: 0.5619
Epoch 646/2000
5/5 [==============================] - 0s 844us/step - loss: 0.9300 - accuracy: 0.5619
Epoch 647/2000
5/5 [==============================] - 0s 769us/step - loss: 0.9414 - accuracy: 0.5619
Epoch 648/2000
5/5 [==============================] - 0s 876us/step - loss: 0.9291 - accuracy: 0.5619
Epoch 649/2000
5/5 [==============================] - 0s 821us/step - loss: 0.9305 - accuracy: 0.5619
Epoch 650/2000
5/5 [==============================] - 0s 788us/step - loss: 0.9344 - accuracy: 0.5619
Epoch 651/2000
5/5 [==============================] - 0s 826us/step - loss: 0.9312 - accuracy: 0.

...

5/5 [==============================] - 0s 804us/step - loss: 0.9011 - accuracy: 0.5592
Epoch 1230/2000
5/5 [==============================] - 0s 833us/step - loss: 0.9172 - accuracy: 0.5545
Epoch 1231/2000
5/5 [==============================] - 0s 812us/step - loss: 0.8925 - accuracy: 0.5782
Epoch 1232/2000
5/5 [==============================] - 0s 784us/step - loss: 0.9125 - accuracy: 0.5687
Epoch 1233/2000
5/5 [==============================] - 0s 761us/step - loss: 0.9037 - accuracy: 0.5640
Epoch 1234/2000
5/5 [==============================] - 0s 808us/step - loss: 0.8920 - accuracy: 0.5545
Epoch 1235/2000
5/5 [==============================] - 0s 832us/step - loss: 0.9151 - accuracy: 0.5498
Epoch 1236/2000
5/5 [==============================] - 0s 842us/step - loss: 0.9186 - accuracy: 0.5640
Epoch 1237/2000
5/5 [==============================] - 0s 815us/step - loss: 0.9316 - accuracy: 0.5403
Epoch 1238/2000
5/5 [==============================] - 0s 910us/step - loss: 0.9195 - acc

5/5 [==============================] - 0s 866us/step - loss: 0.9274 - accuracy: 0.5592
Epoch 1388/2000
5/5 [==============================] - 0s 816us/step - loss: 0.9391 - accuracy: 0.5545
Epoch 1389/2000
5/5 [==============================] - 0s 780us/step - loss: 0.9204 - accuracy: 0.5308
Epoch 1390/2000
5/5 [==============================] - 0s 768us/step - loss: 0.9119 - accuracy: 0.5355
Epoch 1391/2000
5/5 [==============================] - 0s 864us/step - loss: 0.9398 - accuracy: 0.5592
Epoch 1392/2000
5/5 [==============================] - 0s 800us/step - loss: 0.9184 - accuracy: 0.5498
Epoch 1393/2000
5/5 [==============================] - 0s 800us/step - loss: 0.9145 - accuracy: 0.5592
Epoch 1394/2000
5/5 [==============================] - 0s 824us/step - loss: 0.9038 - accuracy: 0.5166
Epoch 1395/2000
5/5 [==============================] - 0s 800us/step - loss: 0.9279 - accuracy: 0.5640
Epoch 1396/2000
5/5 [==============================] - 0s 819us/step - loss: 0.9416 - acc

5/5 [==============================] - 0s 848us/step - loss: 0.9229 - accuracy: 0.5545
Epoch 1546/2000
5/5 [==============================] - 0s 836us/step - loss: 0.9044 - accuracy: 0.5782
Epoch 1547/2000
5/5 [==============================] - 0s 885us/step - loss: 0.9179 - accuracy: 0.5403
Epoch 1548/2000
5/5 [==============================] - 0s 830us/step - loss: 0.9107 - accuracy: 0.5640
Epoch 1549/2000
5/5 [==============================] - 0s 859us/step - loss: 0.8912 - accuracy: 0.5687
Epoch 1550/2000
5/5 [==============================] - 0s 855us/step - loss: 0.9212 - accuracy: 0.5640
Epoch 1551/2000
5/5 [==============================] - 0s 837us/step - loss: 0.8889 - accuracy: 0.5450
Epoch 1552/2000
5/5 [==============================] - 0s 802us/step - loss: 0.9007 - accuracy: 0.5972
Epoch 1553/2000
5/5 [==============================] - 0s 838us/step - loss: 0.9266 - accuracy: 0.5545
Epoch 1554/2000
5/5 [==============================] - 0s 777us/step - loss: 0.9014 - acc

5/5 [==============================] - 0s 809us/step - loss: 0.9127 - accuracy: 0.5640
Epoch 1704/2000
5/5 [==============================] - 0s 866us/step - loss: 0.8784 - accuracy: 0.5545
Epoch 1705/2000
5/5 [==============================] - 0s 866us/step - loss: 0.8991 - accuracy: 0.5735
Epoch 1706/2000
5/5 [==============================] - 0s 793us/step - loss: 0.9178 - accuracy: 0.5545
Epoch 1707/2000
5/5 [==============================] - 0s 792us/step - loss: 0.9184 - accuracy: 0.5640
Epoch 1708/2000
5/5 [==============================] - 0s 839us/step - loss: 0.9141 - accuracy: 0.5687
Epoch 1709/2000
5/5 [==============================] - 0s 827us/step - loss: 0.8982 - accuracy: 0.5782
Epoch 1710/2000
5/5 [==============================] - 0s 830us/step - loss: 0.9139 - accuracy: 0.5592
Epoch 1711/2000
5/5 [==============================] - 0s 843us/step - loss: 0.9090 - accuracy: 0.5687
Epoch 1712/2000
5/5 [==============================] - 0s 863us/step - loss: 0.9152 - acc

5/5 [==============================] - 0s 821us/step - loss: 0.8969 - accuracy: 0.5735
Epoch 1862/2000
5/5 [==============================] - 0s 792us/step - loss: 0.8941 - accuracy: 0.5640
Epoch 1863/2000
5/5 [==============================] - 0s 828us/step - loss: 0.8868 - accuracy: 0.5782
Epoch 1864/2000
5/5 [==============================] - 0s 811us/step - loss: 0.8711 - accuracy: 0.5640
Epoch 1865/2000
5/5 [==============================] - 0s 808us/step - loss: 0.9368 - accuracy: 0.5498
Epoch 1866/2000
5/5 [==============================] - 0s 813us/step - loss: 0.8979 - accuracy: 0.5545
Epoch 1867/2000
5/5 [==============================] - 0s 848us/step - loss: 0.9101 - accuracy: 0.5498
Epoch 1868/2000
5/5 [==============================] - 0s 802us/step - loss: 0.9116 - accuracy: 0.5877
Epoch 1869/2000
5/5 [==============================] - 0s 839us/step - loss: 0.9092 - accuracy: 0.5687
Epoch 1870/2000
5/5 [==============================] - 0s 907us/step - loss: 0.9115 - acc

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.3)

In [70]:
model = baseline_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1000, batch_size=10, shuffle=True)

Epoch 1/1000
17/17 [==============================] - 1s 8ms/step - loss: 1.2935 - accuracy: 0.2699 - val_loss: 1.0810 - val_accuracy: 0.3944
Epoch 2/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.2373 - accuracy: 0.3436 - val_loss: 1.0752 - val_accuracy: 0.3944
Epoch 3/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.2400 - accuracy: 0.3252 - val_loss: 1.0685 - val_accuracy: 0.4507
Epoch 4/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.1898 - accuracy: 0.3374 - val_loss: 1.0645 - val_accuracy: 0.4648
Epoch 5/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.1726 - accuracy: 0.3558 - val_loss: 1.0620 - val_accuracy: 0.4789
Epoch 6/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.1616 - accuracy: 0.3313 - val_loss: 1.0591 - val_accuracy: 0.4789
Epoch 7/1000
17/17 [==============================] - 0s 2ms/step - loss: 1.1183 - accuracy: 0.3804 - val_loss: 1.0568 - val_accuracy: 0.4789
Epoch 

17/17 [==============================] - 0s 2ms/step - loss: 0.9745 - accuracy: 0.5460 - val_loss: 1.0496 - val_accuracy: 0.4930
Epoch 59/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9723 - accuracy: 0.5583 - val_loss: 1.0482 - val_accuracy: 0.4930
Epoch 60/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9717 - accuracy: 0.5521 - val_loss: 1.0495 - val_accuracy: 0.4930
Epoch 61/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9707 - accuracy: 0.5460 - val_loss: 1.0502 - val_accuracy: 0.4930
Epoch 62/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9572 - accuracy: 0.5521 - val_loss: 1.0516 - val_accuracy: 0.4930
Epoch 63/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9849 - accuracy: 0.5399 - val_loss: 1.0537 - val_accuracy: 0.4930
Epoch 64/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9786 - accuracy: 0.5583 - val_loss: 1.0561 - val_accuracy: 0.4930
Epoch 65/1000

17/17 [==============================] - 0s 2ms/step - loss: 0.9577 - accuracy: 0.5521 - val_loss: 1.0606 - val_accuracy: 0.4789
Epoch 116/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9717 - accuracy: 0.5583 - val_loss: 1.0630 - val_accuracy: 0.4930
Epoch 117/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9632 - accuracy: 0.5644 - val_loss: 1.0660 - val_accuracy: 0.4930
Epoch 118/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9517 - accuracy: 0.5521 - val_loss: 1.0663 - val_accuracy: 0.4789
Epoch 119/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9754 - accuracy: 0.5521 - val_loss: 1.0655 - val_accuracy: 0.4789
Epoch 120/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9638 - accuracy: 0.5583 - val_loss: 1.0682 - val_accuracy: 0.4789
Epoch 121/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9518 - accuracy: 0.5583 - val_loss: 1.0702 - val_accuracy: 0.4789
Epoch 1

Epoch 172/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9526 - accuracy: 0.5706 - val_loss: 1.0820 - val_accuracy: 0.4789
Epoch 173/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9657 - accuracy: 0.5521 - val_loss: 1.0850 - val_accuracy: 0.4789
Epoch 174/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9398 - accuracy: 0.5583 - val_loss: 1.0853 - val_accuracy: 0.4789
Epoch 175/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9583 - accuracy: 0.5583 - val_loss: 1.0899 - val_accuracy: 0.4789
Epoch 176/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9392 - accuracy: 0.5644 - val_loss: 1.0995 - val_accuracy: 0.4789
Epoch 177/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9379 - accuracy: 0.5644 - val_loss: 1.1029 - val_accuracy: 0.4789
Epoch 178/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9659 - accuracy: 0.5521 - val_loss: 1.1036 - val_accuracy:

Epoch 229/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9358 - accuracy: 0.5644 - val_loss: 1.1098 - val_accuracy: 0.4789
Epoch 230/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9581 - accuracy: 0.5583 - val_loss: 1.1065 - val_accuracy: 0.4789
Epoch 231/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9429 - accuracy: 0.5583 - val_loss: 1.1067 - val_accuracy: 0.4789
Epoch 232/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9390 - accuracy: 0.5583 - val_loss: 1.1062 - val_accuracy: 0.4789
Epoch 233/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9542 - accuracy: 0.5583 - val_loss: 1.1048 - val_accuracy: 0.4789
Epoch 234/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9456 - accuracy: 0.5460 - val_loss: 1.1054 - val_accuracy: 0.4789
Epoch 235/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9470 - accuracy: 0.5583 - val_loss: 1.1050 - val_accuracy:

...

Epoch 742/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9285 - accuracy: 0.5767 - val_loss: 1.1640 - val_accuracy: 0.4648
Epoch 743/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9245 - accuracy: 0.5521 - val_loss: 1.1668 - val_accuracy: 0.4648
Epoch 744/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9171 - accuracy: 0.5644 - val_loss: 1.1706 - val_accuracy: 0.4648
Epoch 745/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9379 - accuracy: 0.5583 - val_loss: 1.1706 - val_accuracy: 0.4789
Epoch 746/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9435 - accuracy: 0.5767 - val_loss: 1.1639 - val_accuracy: 0.4789
Epoch 747/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9353 - accuracy: 0.5706 - val_loss: 1.1674 - val_accuracy: 0.4789
Epoch 748/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9418 - accuracy: 0.5460 - val_loss: 1.1672 - val_accuracy:

Epoch 799/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9325 - accuracy: 0.5644 - val_loss: 1.1690 - val_accuracy: 0.4648
Epoch 800/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9426 - accuracy: 0.5767 - val_loss: 1.1672 - val_accuracy: 0.4648
Epoch 801/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9197 - accuracy: 0.5521 - val_loss: 1.1696 - val_accuracy: 0.4648
Epoch 802/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9434 - accuracy: 0.5951 - val_loss: 1.1695 - val_accuracy: 0.4648
Epoch 803/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9227 - accuracy: 0.5644 - val_loss: 1.1679 - val_accuracy: 0.4648
Epoch 804/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9140 - accuracy: 0.5828 - val_loss: 1.1672 - val_accuracy: 0.4648
Epoch 805/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9374 - accuracy: 0.5521 - val_loss: 1.1671 - val_accuracy:

Epoch 856/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9493 - accuracy: 0.5399 - val_loss: 1.1598 - val_accuracy: 0.4648
Epoch 857/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9340 - accuracy: 0.5706 - val_loss: 1.1592 - val_accuracy: 0.4648
Epoch 858/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9380 - accuracy: 0.5583 - val_loss: 1.1581 - val_accuracy: 0.4648
Epoch 859/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9517 - accuracy: 0.5706 - val_loss: 1.1571 - val_accuracy: 0.4648
Epoch 860/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9283 - accuracy: 0.5828 - val_loss: 1.1527 - val_accuracy: 0.4648
Epoch 861/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9248 - accuracy: 0.5951 - val_loss: 1.1512 - val_accuracy: 0.4648
Epoch 862/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9359 - accuracy: 0.5644 - val_loss: 1.1489 - val_accuracy:

Epoch 913/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9112 - accuracy: 0.6074 - val_loss: 1.1595 - val_accuracy: 0.4648
Epoch 914/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9509 - accuracy: 0.5337 - val_loss: 1.1586 - val_accuracy: 0.4648
Epoch 915/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9207 - accuracy: 0.5828 - val_loss: 1.1618 - val_accuracy: 0.4648
Epoch 916/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9394 - accuracy: 0.5583 - val_loss: 1.1609 - val_accuracy: 0.4648
Epoch 917/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9301 - accuracy: 0.5583 - val_loss: 1.1688 - val_accuracy: 0.4507
Epoch 918/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9573 - accuracy: 0.5767 - val_loss: 1.1798 - val_accuracy: 0.4648
Epoch 919/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9332 - accuracy: 0.5583 - val_loss: 1.1814 - val_accuracy:

Epoch 970/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9064 - accuracy: 0.5644 - val_loss: 1.1610 - val_accuracy: 0.4648
Epoch 971/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9373 - accuracy: 0.5706 - val_loss: 1.1635 - val_accuracy: 0.4648
Epoch 972/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9307 - accuracy: 0.5521 - val_loss: 1.1612 - val_accuracy: 0.4648
Epoch 973/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9358 - accuracy: 0.5706 - val_loss: 1.1607 - val_accuracy: 0.4648
Epoch 974/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9153 - accuracy: 0.5890 - val_loss: 1.1520 - val_accuracy: 0.4648
Epoch 975/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9360 - accuracy: 0.5583 - val_loss: 1.1556 - val_accuracy: 0.4648
Epoch 976/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9382 - accuracy: 0.5828 - val_loss: 1.1566 - val_accuracy:

##  Implementation #1(a): Binary class on Dataset#1

In [82]:
dataset.manual_label.value_counts()

 1.0    128
 0.0     59
-1.0     47
Name: manual_label, dtype: int64

In [84]:
dataset_binary = dataset[dataset.manual_label != -1]

In [110]:
dataset_binary.describe()

,manual_label,distance,outlier_flag,avg_polarity,sentiment_rate_unsupervised,label_unsupervised
count,187.000000,187.000000,187.000000,187.000000,187.000000,187.0
mean,0.684492,14.315508,0.935829,0.098381,14007.589777,1.0
std,0.465965,9.222572,0.245715,0.184640,53011.189053,0.0
min,0.000000,1.000000,0.000000,-0.333333,471.227408,1.0
25%,0.000000,8.000000,1.000000,0.000000,2458.917784,1.0
50%,1.000000,13.000000,1.000000,0.035714,4063.816272,1.0
75%,1.000000,19.500000,1.000000,0.187051,7642.549283,1.0
max,1.000000,45.000000,1.000000,0.600000,590452.710000,1.0


In [101]:
model_binary = binary_model()

In [89]:
X = dataset_binary[['distance', 'avg_polarity', 'sentiment_rate_unsupervised', 'outlier_flag']].values

In [90]:
y = dataset_binary.manual_label.values

In [97]:
scalar = MinMaxScaler()
scalar.fit(X)
X = scalar.transform(X)

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [102]:
model_binary.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=500, batch_size=5, shuffle=True)

Epoch 1/500
26/26 [==============================] - 0s 4ms/step - loss: 0.8383 - accuracy: 0.3077 - val_loss: 0.7889 - val_accuracy: 0.3509
Epoch 2/500
26/26 [==============================] - 0s 1ms/step - loss: 0.7935 - accuracy: 0.3308 - val_loss: 0.7553 - val_accuracy: 0.3684
Epoch 3/500
26/26 [==============================] - 0s 1ms/step - loss: 0.7555 - accuracy: 0.3462 - val_loss: 0.7238 - val_accuracy: 0.3684
Epoch 4/500
26/26 [==============================] - 0s 1ms/step - loss: 0.7204 - accuracy: 0.3538 - val_loss: 0.6975 - val_accuracy: 0.4386
Epoch 5/500
26/26 [==============================] - 0s 1ms/step - loss: 0.6894 - accuracy: 0.4923 - val_loss: 0.6731 - val_accuracy: 0.6316
Epoch 6/500
26/26 [==============================] - 0s 1ms/step - loss: 0.6634 - accuracy: 0.6769 - val_loss: 0.6542 - val_accuracy: 0.6491
Epoch 7/500
26/26 [==============================] - 0s 1ms/step - loss: 0.6420 - accuracy: 0.7000 - val_loss: 0.6441 - val_accuracy: 0.6491
Epoch 8/500
2

Epoch 59/500
26/26 [==============================] - 0s 1ms/step - loss: 0.6019 - accuracy: 0.7000 - val_loss: 0.6451 - val_accuracy: 0.6491
Epoch 60/500
26/26 [==============================] - 0s 1ms/step - loss: 0.6018 - accuracy: 0.7000 - val_loss: 0.6458 - val_accuracy: 0.6491
Epoch 61/500
26/26 [==============================] - 0s 1ms/step - loss: 0.6015 - accuracy: 0.7000 - val_loss: 0.6460 - val_accuracy: 0.6491
Epoch 62/500
26/26 [==============================] - 0s 1ms/step - loss: 0.6013 - accuracy: 0.7000 - val_loss: 0.6456 - val_accuracy: 0.6491
Epoch 63/500
26/26 [==============================] - 0s 1ms/step - loss: 0.6015 - accuracy: 0.7000 - val_loss: 0.6455 - val_accuracy: 0.6491
Epoch 64/500
26/26 [==============================] - 0s 1ms/step - loss: 0.6015 - accuracy: 0.7000 - val_loss: 0.6454 - val_accuracy: 0.6491
Epoch 65/500
26/26 [==============================] - 0s 1ms/step - loss: 0.6010 - accuracy: 0.7000 - val_loss: 0.6465 - val_accuracy: 0.6491
Epoch 

26/26 [==============================] - 0s 1ms/step - loss: 0.5925 - accuracy: 0.7000 - val_loss: 0.6542 - val_accuracy: 0.6491
Epoch 117/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5919 - accuracy: 0.7000 - val_loss: 0.6542 - val_accuracy: 0.6491
Epoch 118/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5918 - accuracy: 0.7000 - val_loss: 0.6545 - val_accuracy: 0.6491
Epoch 119/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5915 - accuracy: 0.7000 - val_loss: 0.6545 - val_accuracy: 0.6491
Epoch 120/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5909 - accuracy: 0.7000 - val_loss: 0.6550 - val_accuracy: 0.6491
Epoch 121/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5910 - accuracy: 0.7000 - val_loss: 0.6553 - val_accuracy: 0.6491
Epoch 122/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5919 - accuracy: 0.7000 - val_loss: 0.6571 - val_accuracy: 0.6491
Epoch 123/500

26/26 [==============================] - 0s 1ms/step - loss: 0.5808 - accuracy: 0.7000 - val_loss: 0.6703 - val_accuracy: 0.6491
Epoch 174/500
26/26 [==============================] - 0s 2ms/step - loss: 0.5806 - accuracy: 0.7000 - val_loss: 0.6731 - val_accuracy: 0.6491
Epoch 175/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5810 - accuracy: 0.7000 - val_loss: 0.6738 - val_accuracy: 0.6491
Epoch 176/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5800 - accuracy: 0.7000 - val_loss: 0.6726 - val_accuracy: 0.6491
Epoch 177/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5799 - accuracy: 0.7000 - val_loss: 0.6726 - val_accuracy: 0.6491
Epoch 178/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5792 - accuracy: 0.7000 - val_loss: 0.6732 - val_accuracy: 0.6491
Epoch 179/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5807 - accuracy: 0.7000 - val_loss: 0.6728 - val_accuracy: 0.6491
Epoch 180/500

26/26 [==============================] - 0s 1ms/step - loss: 0.5702 - accuracy: 0.7000 - val_loss: 0.6939 - val_accuracy: 0.6491
Epoch 231/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5696 - accuracy: 0.7000 - val_loss: 0.6933 - val_accuracy: 0.6491
Epoch 232/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5711 - accuracy: 0.7000 - val_loss: 0.6974 - val_accuracy: 0.6491
Epoch 233/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5699 - accuracy: 0.7000 - val_loss: 0.6966 - val_accuracy: 0.6491
Epoch 234/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5702 - accuracy: 0.7000 - val_loss: 0.6948 - val_accuracy: 0.6491
Epoch 235/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5694 - accuracy: 0.7000 - val_loss: 0.6981 - val_accuracy: 0.6491
Epoch 236/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5700 - accuracy: 0.7000 - val_loss: 0.6963 - val_accuracy: 0.6491
Epoch 237/500

26/26 [==============================] - 0s 1ms/step - loss: 0.5680 - accuracy: 0.7077 - val_loss: 0.7174 - val_accuracy: 0.6316
Epoch 288/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5662 - accuracy: 0.7154 - val_loss: 0.7259 - val_accuracy: 0.6491
Epoch 289/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5631 - accuracy: 0.7077 - val_loss: 0.7220 - val_accuracy: 0.6316
Epoch 290/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5638 - accuracy: 0.7077 - val_loss: 0.7203 - val_accuracy: 0.6316
Epoch 291/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5632 - accuracy: 0.7000 - val_loss: 0.7222 - val_accuracy: 0.6316
Epoch 292/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5665 - accuracy: 0.7077 - val_loss: 0.7205 - val_accuracy: 0.6316
Epoch 293/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5655 - accuracy: 0.6923 - val_loss: 0.7249 - val_accuracy: 0.6491
Epoch 294/500

26/26 [==============================] - 0s 1ms/step - loss: 0.5604 - accuracy: 0.7077 - val_loss: 0.7410 - val_accuracy: 0.6316
Epoch 345/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5599 - accuracy: 0.7077 - val_loss: 0.7400 - val_accuracy: 0.6316
Epoch 346/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5620 - accuracy: 0.7077 - val_loss: 0.7443 - val_accuracy: 0.6491
Epoch 347/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5598 - accuracy: 0.7077 - val_loss: 0.7397 - val_accuracy: 0.6316
Epoch 348/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5593 - accuracy: 0.6923 - val_loss: 0.7411 - val_accuracy: 0.6316
Epoch 349/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5595 - accuracy: 0.7000 - val_loss: 0.7413 - val_accuracy: 0.6316
Epoch 350/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5591 - accuracy: 0.7077 - val_loss: 0.7437 - val_accuracy: 0.6316
Epoch 351/500

26/26 [==============================] - 0s 1ms/step - loss: 0.5564 - accuracy: 0.7000 - val_loss: 0.7531 - val_accuracy: 0.6316
Epoch 402/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5558 - accuracy: 0.6923 - val_loss: 0.7541 - val_accuracy: 0.6316
Epoch 403/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5560 - accuracy: 0.6923 - val_loss: 0.7549 - val_accuracy: 0.6316
Epoch 404/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5579 - accuracy: 0.7000 - val_loss: 0.7516 - val_accuracy: 0.6491
Epoch 405/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5554 - accuracy: 0.6923 - val_loss: 0.7551 - val_accuracy: 0.6316
Epoch 406/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5570 - accuracy: 0.7077 - val_loss: 0.7559 - val_accuracy: 0.6316
Epoch 407/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5558 - accuracy: 0.7000 - val_loss: 0.7515 - val_accuracy: 0.6491
Epoch 408/500

26/26 [==============================] - 0s 1ms/step - loss: 0.5560 - accuracy: 0.7077 - val_loss: 0.7590 - val_accuracy: 0.6316
Epoch 459/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5530 - accuracy: 0.7000 - val_loss: 0.7619 - val_accuracy: 0.6316
Epoch 460/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5534 - accuracy: 0.6846 - val_loss: 0.7648 - val_accuracy: 0.6316
Epoch 461/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5533 - accuracy: 0.7077 - val_loss: 0.7628 - val_accuracy: 0.6316
Epoch 462/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5539 - accuracy: 0.7000 - val_loss: 0.7612 - val_accuracy: 0.6316
Epoch 463/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5535 - accuracy: 0.7000 - val_loss: 0.7629 - val_accuracy: 0.6491
Epoch 464/500
26/26 [==============================] - 0s 1ms/step - loss: 0.5529 - accuracy: 0.7000 - val_loss: 0.7624 - val_accuracy: 0.6491
Epoch 465/500